In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

# Paths
PROJECT_ROOT = Path(r"C:\Users\sdnea\Dev\permit-data-dev")
DB_PATH = PROJECT_ROOT / "permit_data.db"

# Connect
conn = sqlite3.connect(DB_PATH)


In [2]:
bronze_tables = ["permits_2024_bronze", "permits_2025_bronze"]

dfs = {}
for table in bronze_tables:
    dfs[table] = pd.read_sql(f"SELECT * FROM {table}", conn)
    print(f"{table}: {dfs[table].shape[0]} rows, {dfs[table].shape[1]} cols")


permits_2024_bronze: 1411 rows, 10 cols
permits_2025_bronze: 1124 rows, 10 cols


In [3]:
for table, df in dfs.items():
    print(f"\n{table} columns:")
    print(list(df.columns))



permits_2024_bronze columns:
['Record #', 'Record Type', 'Title', 'Type of Improvement', 'Date Issued', 'Type of Property', 'Full Address', 'Estimated Project Value', 'Contractor Used', 'Project Description']

permits_2025_bronze columns:
['Record #', 'Record Type', 'Title', 'Type of Improvement', 'Date Issued', 'Type of Property', 'Full Address', 'Estimated Project Value', 'Contractor Used', 'Project Description']


In [4]:
def standardize_2024_2025(df):
    return df.rename(columns={
        "Record #": "record_id",
        "Record Type": "record_type",
        "Title": "title",
        "Type of Improvement": "permit_type",
        "Date Issued": "date_issued",
        "Type of Property": "type_of_property",
        "Full Address": "address",
        "Estimated Project Value": "valuation",
        "Contractor Used": "contractor",
        "Project Description": "description"
    }).assign(
        total_fees_paid=None,
        trade_contractors=None
    )


In [5]:
dfs_std = {
    "permits_2024_bronze": standardize_2024_2025(dfs["permits_2024_bronze"]),
    "permits_2025_bronze": standardize_2024_2025(dfs["permits_2025_bronze"]),
}


In [6]:
permits_silver = pd.concat(dfs_std.values(), ignore_index=True)
print(f"Unified permits_silver: {permits_silver.shape[0]} rows, {permits_silver.shape[1]} cols")
print("Columns:", list(permits_silver.columns))


Unified permits_silver: 2535 rows, 12 cols
Columns: ['record_id', 'record_type', 'title', 'permit_type', 'date_issued', 'type_of_property', 'address', 'valuation', 'contractor', 'description', 'total_fees_paid', 'trade_contractors']


In [7]:
permits_silver.to_sql("permits_silver", conn, if_exists="replace", index=False)
print("✅ Created permits_silver table in SQLite")

print("\nBefore conversions:\n", permits_silver.dtypes)

# Convert valuation to numeric
permits_silver["valuation"] = pd.to_numeric(permits_silver["valuation"], errors="coerce")

# Convert total_fees_paid to numeric (will just be NaN since not in 2024/25)
permits_silver["total_fees_paid"] = pd.to_numeric(permits_silver["total_fees_paid"], errors="coerce")

# Convert date_issued to datetime
permits_silver["date_issued"] = pd.to_datetime(permits_silver["date_issued"], errors="coerce")

print("\nAfter conversions:\n", permits_silver.dtypes)

# Preview top 10
permits_silver.head(10)


✅ Created permits_silver table in SQLite

Before conversions:
 record_id             object
record_type           object
title                 object
permit_type           object
date_issued           object
type_of_property      object
address               object
valuation            float64
contractor            object
description           object
total_fees_paid       object
trade_contractors     object
dtype: object

After conversions:
 record_id                    object
record_type                  object
title                        object
permit_type                  object
date_issued          datetime64[ns]
type_of_property             object
address                      object
valuation                   float64
contractor                   object
description                  object
total_fees_paid             float64
trade_contractors            object
dtype: object


,record_id,record_type,title,permit_type,date_issued,type_of_property,address,valuation,contractor,description,total_fees_paid,trade_contractors
0,PLMB-23-13,Plumbing Permit,Residential Plumbing Permit,Water Heater,2024-01-02 20:22:00,Residential,"146 ASHLAND AVE SW, NORTH CANTON, OH 44709",1600.00,By-tor Contracting Inc,Install/Exchange water heater,NaN,None
1,PLMB-23-12,Plumbing Permit,Residential Plumbing Permit,Water Heater,2024-01-02 20:23:00,Residential,"1681 WESTFIELD AVE SW, NORTH CANTON, OH 44709",1600.00,By-tor Contracting Inc,Install/Exchange Water Heater,NaN,None
2,PLMB-24-3,Plumbing Permit,Residential Plumbing Permit,Water Heater,2024-01-02 21:06:00,Residential,"1161 N MAIN ST, Unit 19, NORTH CANTON, OH 44720",1100.00,H. Jack's Plumbing & Heating Co.,Water heater replacement,NaN,None
3,PLMB-24-1,Plumbing Permit,Residential Plumbing Permit,New Construction/Addition,2024-01-02 21:27:00,Residential,"1290 LONGSHADOW AVE NE, NORTH CANTON, OH 44720",18000.00,D & A Plumbing and Heating,PLUMBING FOR NEW CONSTRUCTION HOME,NaN,None
4,PLMB-24-2,Plumbing Permit,Residential Plumbing Permit,Alteration/Remodel,2024-01-03 17:03:00,Residential,"514 EVERHARD RD SW, NORTH CANTON, OH 44709",6000.00,CLEVELAND ROOTER LLC DBA MR ROOTER PLUMBING,APPROX 5-10' ON EXTERIOR SANITARY SEWER REPAIR,NaN,None
5,PLMB-23-11,Plumbing Permit,Residential Plumbing Permit,Other,2024-01-03 17:15:00,Residential,"705 APPLEGROVE ST NW, Unit 709, NORTH CANTON, ...",165.00,That Plumber,Check gas valves,NaN,None
6,PLMB-24-4,Plumbing Permit,Residential Plumbing Permit,Water Heater,2024-01-04 16:26:00,Residential,"28 LASALLE CT SE, NORTH CANTON, OH 44709",6150.00,ComfortPro Heating and Cooling LLC,REPLACE TANK WATER HEATER WITH TANKLESS WATER ...,NaN,None
7,PLMB-24-5,Plumbing Permit,Residential Plumbing Permit,Natural Gas Line,2024-01-05 16:27:00,Residential,"389 ORCHARD AVE NE, Unit 391, NORTH CANTON, OH...",1500.00,Coblentz Plumbing Solutions,Replacement of in-line gas valve,NaN,None
8,ROOF-24-1,Roof Permit,Residential Roof Permit,New/ Re-Roof,2024-01-05 18:29:00,Residential,"140 JAMES ST SW, Unit 142, NORTH CANTON, OH 44720",12000.00,M D Home Services Inc,Re-Roof,NaN,None
9,PLMB-24-6,Plumbing Permit,Residential Plumbing Permit,Water Heater,2024-01-08 15:38:00,Residential,"179 APPLEGROVE ST NE, Unit B6, NORTH CANTON, O...",1507.98,"JEM Plumbing, INC",Hot Water Tank Replacement,NaN,None


In [8]:
# Kernel 8: Export Silver table to Excel
EXPORT_PATH = PROJECT_ROOT / "silver" / "permits_silver.xlsx"

# Make sure the silver/ folder exists
EXPORT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Write DataFrame to Excel
permits_silver.to_excel(EXPORT_PATH, index=False)

print(f"✅ Exported Silver permits table to {EXPORT_PATH}")


✅ Exported Silver permits table to C:\Users\sdnea\Dev\permit-data-dev\silver\permits_silver.xlsx
